### importing the neccessary libraries

In [32]:


from pyspark.sql.functions import *
from pyspark.sql.types import *




StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 39, Finished, Available, Finished)

### Import the dataset from Bronze Layer

spark.read.format("csv")   -> Specifies that the data is in CSV format and will be read using Spark.

.option("header", "true") -> Indicates that the first row of the file contains column names.

.option("inferSchema", "true") -> Automatically detects the data types of each column.

In [59]:
df = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/movie_analysis_lakehouse.Lakehouse/Files/bronze/Movie_NewDataset.csv")
# df now is a Spark DataFrame containing CSV data from "abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.com/movie_analysis_lakehouse.Lakehouse/Files/bronze/Movie_NewDataset.csv".
display(df)

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 66, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cb732617-4a83-48a2-8ea2-f50dd74bd1e2)

### checking if there is null values in the dataset

col(c).isNull() ==Checks if a value in column c is NULL (missing).

.cast("int")==Converts True (which represents a NULL value) to 1, and False to 0.

sum(col(c).isNull().cast("int"))==Sums up the 1s for each column to count the total number of NULL values.

.alias(c)===Gives the count result the same column name for display.

df.select([...]).show()==Selects and displays the NULL value counts for all columns in a tabular format.

In [60]:


df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]).show()

## There is no null values in  the dataset 



StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 67, Finished, Available, Finished)

+-------+--------+------+---------+-----------+------+------------+
|user_id|movie_id|rating|timestamp|movie_title|genres|release_year|
+-------+--------+------+---------+-----------+------+------------+
|      0|       0|     0|        0|          0|     0|           0|
+-------+--------+------+---------+-----------+------+------------+



c.lower() --> Converts each column name to lowercase for consistency.

.replace(" ", "_")-->Replaces spaces with underscores _ to ensure compatibility (especially useful when working with SQL and PySpark).

df.toDF(*[...])-->Renames the columns in df with the modified names (lowercased and formatted).

df.printSchema()

Prints the schema of the updated DataFrame to confirm the changes.

In [61]:

from pyspark.sql.functions import col

# Standardizing column names (lowercase & replacing spaces)
df =df.toDF(*[c.lower().replace(" ", "_") for c in df.columns])
df.printSchema()


StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 68, Finished, Available, Finished)

root
 |-- user_id: integer (nullable = true)
 |-- movie_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- release_year: integer (nullable = true)



In [95]:
df.count()   # counting the data count

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 102, Finished, Available, Finished)

148

In [ ]:
df.dropDuplicates() # remove duplicates

In [62]:
# Count duplicate rows 
duplicates_count = df.count() - df.dropDuplicates().count()
print(duplicates_count)

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 69, Finished, Available, Finished)

0


In [51]:
df.columns  # Retriving the col names 

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 58, Finished, Available, Finished)

['movie_id',
 'user_id',
 'rating',
 'timestamp',
 'movie_title',
 'genres',
 'release_year',
 'avg_ratings',
 'avg_ratings']

In [63]:
df.summary()

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 70, Finished, Available, Finished)

DataFrame[summary: string, user_id: string, movie_id: string, rating: string, timestamp: string, movie_title: string, genres: string, release_year: string]

In [64]:
df.describe()

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 71, Finished, Available, Finished)

DataFrame[summary: string, user_id: string, movie_id: string, rating: string, timestamp: string, movie_title: string, genres: string, release_year: string]

In [65]:
df.printSchema()  # Gives the Data Types of the columns

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 72, Finished, Available, Finished)

root
 |-- user_id: integer (nullable = true)
 |-- movie_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- release_year: integer (nullable = true)



In [55]:
# Convert data types using withcolumn and cast functions
# df = df.withColumn("timestamp", col("timestamp").cast("timestamp"))

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 62, Finished, Available, Finished)

In [66]:
df.printSchema()

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 73, Finished, Available, Finished)

root
 |-- user_id: integer (nullable = true)
 |-- movie_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- release_year: integer (nullable = true)



In [67]:
df.show(10)  # this will show the first 10 rows of the dataset

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 74, Finished, Available, Finished)

+-------+--------+------+----------------+-------------+-------------------+------------+
|user_id|movie_id|rating|       timestamp|  movie_title|             genres|release_year|
+-------+--------+------+----------------+-------------+-------------------+------------+
|    179|    5019|     3|24-03-2025 10:00|The Godfather|        Crime|Drama|        1972|
|    131|    5002|     5|24-03-2025 10:01|   The Matrix|      Action|Sci-Fi|        1999|
|    124|    5019|     5|24-03-2025 10:02|The Godfather|        Crime|Drama|        1972|
|    187|    5013|     4|24-03-2025 10:03| Finding Nemo|Animation|Adventure|        2003|
|    172|    5019|     2|24-03-2025 10:04|The Godfather|        Crime|Drama|        1972|
|    136|    5002|     3|24-03-2025 10:09|   The Matrix|      Action|Sci-Fi|        1999|
|    102|    5013|     2|24-03-2025 10:23| Finding Nemo|Animation|Adventure|        2003|
|    146|    5002|     3|24-03-2025 10:27|   The Matrix|      Action|Sci-Fi|        1999|
|    108| 

In [68]:
# here, we find the average ratings for movie id 

avg_rate = df.groupBy("movie_id").agg(avg("rating").alias('avg_ratings'))
avg_rate.show()


## Groups the dataset by movie_id, meaning all ratings for the same movie will be grouped together.
## avg("rating") → Calculates the average rating for each movie_id.

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 75, Finished, Available, Finished)

+--------+-----------------+
|movie_id|      avg_ratings|
+--------+-----------------+
|    5013|3.152173913043478|
|    5002|             3.38|
|    5019|3.269230769230769|
+--------+-----------------+



In [69]:
df=df.join(avg_rate, on="movie_id", how="left")
df.show(10)  


# Run this query only once 
# This will show  avg_rate column in the original dataset


StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 76, Finished, Available, Finished)

+--------+-------+------+----------------+-------------+-------------------+------------+-----------------+
|movie_id|user_id|rating|       timestamp|  movie_title|             genres|release_year|      avg_ratings|
+--------+-------+------+----------------+-------------+-------------------+------------+-----------------+
|    5019|    179|     3|24-03-2025 10:00|The Godfather|        Crime|Drama|        1972|3.269230769230769|
|    5002|    131|     5|24-03-2025 10:01|   The Matrix|      Action|Sci-Fi|        1999|             3.38|
|    5019|    124|     5|24-03-2025 10:02|The Godfather|        Crime|Drama|        1972|3.269230769230769|
|    5013|    187|     4|24-03-2025 10:03| Finding Nemo|Animation|Adventure|        2003|3.152173913043478|
|    5019|    172|     2|24-03-2025 10:04|The Godfather|        Crime|Drama|        1972|3.269230769230769|
|    5002|    136|     3|24-03-2025 10:09|   The Matrix|      Action|Sci-Fi|        1999|             3.38|
|    5013|    102|     2|24-

In [70]:

# Sort movies by average rating (descending order)
df = df.orderBy(col("avg_ratings").desc())

# Show top 10 highest-rated movies
df.show(10)


StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 77, Finished, Available, Finished)

+--------+-------+------+----------------+-----------+-------------+------------+-----------+
|movie_id|user_id|rating|       timestamp|movie_title|       genres|release_year|avg_ratings|
+--------+-------+------+----------------+-----------+-------------+------------+-----------+
|    5002|    169|     4|24-03-2025 13:50| The Matrix|Action|Sci-Fi|        1999|       3.38|
|    5002|    103|     2|24-03-2025 18:37| The Matrix|Action|Sci-Fi|        1999|       3.38|
|    5002|    105|     5|24-03-2025 13:56| The Matrix|Action|Sci-Fi|        1999|       3.38|
|    5002|    131|     5|24-03-2025 10:01| The Matrix|Action|Sci-Fi|        1999|       3.38|
|    5002|    155|     3|24-03-2025 14:12| The Matrix|Action|Sci-Fi|        1999|       3.38|
|    5002|    146|     3|24-03-2025 10:27| The Matrix|Action|Sci-Fi|        1999|       3.38|
|    5002|    103|     2|24-03-2025 17:29| The Matrix|Action|Sci-Fi|        1999|       3.38|
|    5002|    197|     2|24-03-2025 11:45| The Matrix|Action

In [73]:
# Perform a group-by operation to calculate the average rating for movies by release year. 

avg_rating_by_year_df = df.groupBy("release_year").agg(avg("rating").alias("avg_rating_yrwise"))
avg_rating_by_year_df.show()


#groupBy("release_year") --> group all the release yr with avg rating 
 

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 80, Finished, Available, Finished)

+------------+-----------------+
|release_year|avg_rating_yrwise|
+------------+-----------------+
|        2003|3.152173913043478|
|        1972|3.269230769230769|
|        1999|             3.38|
+------------+-----------------+



## Additional Point

df.groupBy("movie_id")

Groups the dataset by movie_id, meaning all ratings for the same movie will be counted together.

.count()

Counts the number of ratings each movie has received.

.orderBy(col("count").desc())

Sorts the movies in descending order based on the count (i.e., most-rated movies appear first).

In [75]:
 ## Most Popular Movies based on Number of Ratings 

popular_movies = df.groupBy("movie_id").count().orderBy(col("count").desc())
popular_movies.show(10)



StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 82, Finished, Available, Finished)

+--------+-----+
|movie_id|count|
+--------+-----+
|    5019|   52|
|    5002|   50|
|    5013|   46|
+--------+-----+



In [77]:
## Find  which users have rated to the  most movies:

active_users = df.groupBy("user_id").count().orderBy(col("count").desc())
active_users.show(10)


StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 84, Finished, Available, Finished)

+-------+-----+
|user_id|count|
+-------+-----+
|    169|    5|
|    172|    5|
|    124|    5|
|    107|    4|
|    105|    4|
|    130|    3|
|    129|    3|
|    175|    3|
|    167|    3|
|    168|    3|
+-------+-----+
only showing top 10 rows



In [78]:
##  how many times each rating was given:

df.groupBy("rating").count().orderBy(col("rating")).show()


StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 85, Finished, Available, Finished)

+------+-----+
|rating|count|
+------+-----+
|     1|   20|
|     2|   29|
|     3|   26|
|     4|   37|
|     5|   36|
+------+-----+



# Save to Silver_Layer

Why mergeSchema=True ?
 Allows new columns to be added when writing.
 Prevents schema mismatch errors when updating an existing Delta table.
 Ensures data consistency across multiple writes.

df.write.format("delta") \
    .option("mergeSchema", "true") \  # Enables schema merging
    .mode("overwrite") \  # Overwrites the existing data while keeping schema updates
    .save("Files/Silver/silver_movie_delta_new")


In [84]:
#  error ---df= df.write.format("delta").mode("overwrite").save("Files/Silver_movie") need= mergeschema 

df.write.format("delta").option("mergeSchema", "true").mode("overwrite").save("Files/Silver/silver_movie_delta_new")
# This will auto-update the schema while keeping existing data.

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 91, Finished, Available, Finished)

In [85]:
lakehouse = spark.read.format("delta").load("Files/Silver/silver_movie_delta_new")

## If your Silver Layer exists.
#If the Silver Layer table already exists, this step ensures that you can access and analyze the data.

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 92, Finished, Available, Finished)

In [87]:
spark.read.format("delta").load("Files/Silver/silver_movie_delta_new").printSchema()
# this will print the schema of the silver delta file 

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 94, Finished, Available, Finished)

root
 |-- release_year: integer (nullable = true)
 |-- movie_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- avg_ratings: double (nullable = true)
 |-- avg_rating_yrwise: double (nullable = true)
 |-- month: integer (nullable = true)



In [88]:
df = spark.read.format("delta").load("Files/Silver/silver_movie_delta_new")
df.show(10)  ## read the data


StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 95, Finished, Available, Finished)

+------------+--------+-------+------+----------------+-------------+-----------+-----------------+-----------------+-----+
|release_year|movie_id|user_id|rating|       timestamp|  movie_title|     genres|      avg_ratings|avg_rating_yrwise|month|
+------------+--------+-------+------+----------------+-------------+-----------+-----------------+-----------------+-----+
|        1972|    5019|    179|     3|24-03-2025 10:00|The Godfather|Crime|Drama|3.269230769230769|3.269230769230769| NULL|
|        1972|    5019|    124|     5|24-03-2025 10:02|The Godfather|Crime|Drama|3.269230769230769|3.269230769230769| NULL|
|        1972|    5019|    172|     2|24-03-2025 10:04|The Godfather|Crime|Drama|3.269230769230769|3.269230769230769| NULL|
|        1972|    5019|    175|     1|24-03-2025 11:05|The Godfather|Crime|Drama|3.269230769230769|3.269230769230769| NULL|
|        1972|    5019|    145|     3|24-03-2025 12:06|The Godfather|Crime|Drama|3.269230769230769|3.269230769230769| NULL|
|       

In [90]:
df = df.drop("avg_rating_yrwise","month")
df.show(5)
 ## Drop unwanted columns 

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 97, Finished, Available, Finished)

+------------+--------+-------+------+----------------+-------------+-----------+-----------------+
|release_year|movie_id|user_id|rating|       timestamp|  movie_title|     genres|      avg_ratings|
+------------+--------+-------+------+----------------+-------------+-----------+-----------------+
|        1972|    5019|    179|     3|24-03-2025 10:00|The Godfather|Crime|Drama|3.269230769230769|
|        1972|    5019|    124|     5|24-03-2025 10:02|The Godfather|Crime|Drama|3.269230769230769|
|        1972|    5019|    172|     2|24-03-2025 10:04|The Godfather|Crime|Drama|3.269230769230769|
|        1972|    5019|    175|     1|24-03-2025 11:05|The Godfather|Crime|Drama|3.269230769230769|
|        1972|    5019|    145|     3|24-03-2025 12:06|The Godfather|Crime|Drama|3.269230769230769|
+------------+--------+-------+------+----------------+-------------+-----------+-----------------+
only showing top 5 rows



In [91]:
# df.write.format("delta").mode("overwrite").save("Files/Silver/silver_movie_delta_new")

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 98, Finished, Available, Finished)

spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")


-- By default (true), Spark checks if a file format is Delta before reading/writing.
 --When set to false, Spark skips this check, which can help if:
---You are migrating non-Delta files to Delta.

--#You have mixed-format data in a folder.
--# it just disables a check. If you are sure your files are Delta, you don’t need to change this setting.

In [92]:
df.write.format("delta").mode("overwrite").saveAsTable("Silver_movie")
# save AS table 

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 99, Finished, Available, Finished)

In [93]:
%%sql  --- use sql query to check if data is loaded to the table 
SELECT * FROM Silver_movie;


StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 100, Finished, Available, Finished)

<Spark SQL result set with 148 rows and 8 fields>

In [94]:
%%sql --- show the tables 
SHOW TABLES;

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 101, Finished, Available, Finished)

<Spark SQL result set with 2 rows and 3 fields>

# Checking Data Before Moving Forward

In [96]:
df.dropDuplicates().count() == df.count()

# There are no duplicate records 


StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 103, Finished, Available, Finished)

True

In [97]:
# cheking if the datatypes are correct or not/

df.printSchema()

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 104, Finished, Available, Finished)

root
 |-- release_year: integer (nullable = true)
 |-- movie_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- avg_ratings: double (nullable = true)



## Saving Data to Gold Layer

In [103]:
df.write.format("delta").mode("overwrite").save("Files/Gold/movie_gold_delta_new")
# save the gold file with new path

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 110, Finished, Available, Finished)

In [100]:
# data types of the gold layer

spark.read.format("delta").load("Files/Gold/movie_gold_delta_new").printSchema()


StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 107, Finished, Available, Finished)

root
 |-- release_year: integer (nullable = true)
 |-- movie_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- avg_ratings: double (nullable = true)



In [104]:
df.write.format("delta").mode("overwrite").saveAsTable("Gold_movie_new")

# save the delta file to the table 

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 111, Finished, Available, Finished)

In [105]:
%%sql
SELECT * FROM gold_movie_new;

StatementMeta(, c8d07ec2-8da2-4da5-8914-e233f25ed4d5, 112, Finished, Available, Finished)

<Spark SQL result set with 148 rows and 8 fields>